In [194]:
import pandas as pd
import numpy as np

In [204]:
def load_movies_dataset():
    movie_data = pd.read_csv(data_dir + './dataset1/movie.csv')
    movie_data = movie_data.rename(columns = {'评分': "豆瓣网评分"})
    return movie_data 

def load_user_and_ratings() :
    user_data = pd.read_csv(data_dir + './dataset1/user.csv')
    user_data['评论时间'] = pd.to_datetime(user_data['评论时间'])   
    return user_data 

#### step one 用户数据集处理

In [239]:
def call_data_process(dfuserin,dfmoviein):
    ur1 = dfuserin.groupby(['用户ID']).评分.agg( {'user_rating_avg':np.mean ,
                                           'user_rating_max':np.max ,
                                           'user_rating_min':np.min ,
                                           'user_rating_median':np.median
                                          }).reset_index()
    print('ur1 shape \n',ur1.shape)
    dfuserin['用户评论次数_观看电影个数'] = dfuserin.groupby(['用户ID'])['评分'].transform('count')
    print('dfuserin shape \n',dfuserin.shape)
    u_data = pd.merge(ur1,dfuserin,on='用户ID',how = 'inner')
    print('u_data shape \n',u_data.shape)
    user_and_movies_df = pd.merge(u_data,movies_df,on=['电影名','类型'],how='inner')
    #print(type(user_and_movies_df))
    return user_and_movies_df

In [245]:
def get_all_real_datas():
    data_dir = 'D:/python/Jupyter_Last_project/dataset/'
    movies_df = load_movies_dataset()
    print('movies_df columns \n',movies_df.columns)
    print('movies_df shape \n',movies_df.shape)
    userdata = load_user_and_ratings()
    print('userdata columns \n',userdata.columns)
    print('userdata shape\n',userdata.shape)

    del userdata['用户名']
    del userdata['评论时间']
    all_df = call_data_process(userdata,movies_df).copy(deep=True)
    all_df = all_df.dropna(how='any', axis=0)
    #print(all_df.columns)
    #print(all_df.shape)
    return all_df

In [246]:
all_real_data = get_all_real_datas()

all_real_data = all_real_data.dropna(how='any', axis=0)
print(all_real_data.shape)

movies_df columns 
 Index(['类型', '主演', '地区', '导演', '特色', '豆瓣网评分', '电影名'], dtype='object')
movies_df shape 
 (89524, 7)
userdata columns 
 Index(['评分', '用户名', '评论时间', '用户ID', '电影名', '类型'], dtype='object')
userdata shape
 (188843, 6)
ur1 shape 
 (13545, 5)
dfuserin shape 
 (188843, 5)
u_data shape 
 (188843, 9)


D:\Anaconda3\envs\py36\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """


(253054, 14)


In [247]:
del all_real_data['主演']

In [248]:
print(all_real_data.shape)

(253054, 13)


In [249]:
all_real_data.columns

Index(['用户ID', 'user_rating_avg', 'user_rating_max', 'user_rating_min',
       'user_rating_median', '评分', '电影名', '类型', '用户评论次数_观看电影个数', '地区', '导演',
       '特色', '豆瓣网评分'],
      dtype='object')

#### step two在原始数据集上做训练  即训练的每条都是用户见过的数据

In [250]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, LabelEncoder
from tensorflow.keras.layers import Input, Embedding, Dense, Flatten, Dropout, SpatialDropout1D, Activation, concatenate
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import ReLU, PReLU, LeakyReLU, ELU
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.utils import shuffle

In [251]:
COLUMNS = [
    '用户ID', 'user_rating_avg', 'user_rating_max', 'user_rating_min',
       'user_rating_median', '评分', '电影名', '用户评论次数_观看电影个数', 
    '类型',  '地区',
       '导演', '特色', '豆瓣网评分'
]

LABEL_COLUMN = "label"

CATEGORICAL_COLUMNS = [
    "类型", "特色","地区", "导演","电影名"
]

CONTINUOUS_COLUMNS = [
    "用户ID", "user_rating_avg", "user_rating_max", "user_rating_min",
    "user_rating_median","用户评论次数_观看电影个数","豆瓣网评分"
]

In [252]:
def preprocessing():
    assert all_real_data.shape[1] == 13
    # 缺失值删除
    all_real_data.dropna(how='any', axis=0)
    # 给数据
    # 标签2值化
    all_data = all_real_data.copy(deep=True)
    all_data[LABEL_COLUMN] = all_data['评分'].apply(lambda x: 1 if x>=6 else 0 )
    all_data.pop('评分')
    # 标签y
    y = all_data[LABEL_COLUMN].values
    print(all_data.label.value_counts())
    #print('y is\n',y)
    # 数据X
    all_data.pop(LABEL_COLUMN)
    # 类别型的label encoding
    for c in CATEGORICAL_COLUMNS:
        le = LabelEncoder()
        all_data[c] = le.fit_transform(all_data[c])
    #随机打乱数据
    all_data = shuffle(all_data)
    # 分别取出train和test的特征和标签
    # 打乱后取90%作为训练集 10%作为测试集
    train_size = int(all_data.shape[0] * 0.9)
    x_train = all_data.iloc[:train_size]
    #print('x_train.shape',x_train.shape)
    y_train = y[:train_size]
    x_test = all_data.iloc[train_size:]
    y_test = y[train_size:]
    # 类别型的列
    x_train_categ = np.array(x_train[CATEGORICAL_COLUMNS])
    #print('type(x_train_categ)\n',type(x_train_categ))
    x_test_categ = np.array(x_test[CATEGORICAL_COLUMNS])
    # 连续值的列
    x_train_conti = np.array(x_train[CONTINUOUS_COLUMNS], dtype='float64')
    x_test_conti = np.array(x_test[CONTINUOUS_COLUMNS], dtype='float64')
    # 对连续值的列做幅度缩放
    scaler = StandardScaler()
    x_train_conti = scaler.fit_transform(x_train_conti)
    x_test_conti = scaler.transform(x_test_conti)
    return [x_train, y_train, x_test, y_test, x_train_categ, x_test_categ, x_train_conti, x_test_conti, all_data]




In [253]:
class Wide_and_Deep:
    def __init__(self, mode='wide and deep'):
        self.mode = mode
        x_train, y_train, x_test, y_test, x_train_categ, x_test_categ, x_train_conti, x_test_conti, all_data \
            = preprocessing()
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        self.x_train_categ = x_train_categ
        self.x_test_categ = x_test_categ
        self.x_train_conti = x_train_conti
        self.x_test_conti = x_test_conti
        self.all_data = all_data
        self.poly = PolynomialFeatures(degree=2, interaction_only=True)
        self.x_train_categ_poly = self.poly.fit_transform(x_train_categ)
        self.x_test_categ_poly = self.poly.transform(x_test_categ)
        self.categ_inputs = None
        self.conti_input = None
        self.deep_component_outlayer = None
        self.logistic_input = None
        self.model = None
        
    #input_length: 输入序列的长度    
    #dim -->  输入的不同的词的个数
    #embed_dim -->  输出维度
    # embed_i shape
    #  (None, 1, 2)
    # flatten_i shape
    #  (None, 2)    
    def deep_component(self):
        # deep部分的组件
        categ_inputs = []
        categ_embeds = []
        # 对类别型的列做embedding
        count = 0
        for i in range(len(CATEGORICAL_COLUMNS)):
            input_i = Input(shape=(1,), dtype='int32')
            dim = len(np.unique(self.all_data[CATEGORICAL_COLUMNS[i]]))   
            embed_dim = int(np.ceil(dim ** 0.25))
            embed_i = Embedding(dim, embed_dim, input_length=1)(input_i)
            flatten_i = Flatten()(embed_i)
            categ_inputs.append(input_i)     
            categ_embeds.append(flatten_i)
        # 连续值的列
        conti_input = Input(shape=(len(CONTINUOUS_COLUMNS),))
        conti_dense = Dense(256, use_bias=False)(conti_input)
        # 拼接类别型的embedding和连续值特征
        concat_embeds = concatenate([conti_dense]+categ_embeds)
        # 激活层与BN层
        concat_embeds = Activation('relu')(concat_embeds)
        bn_concat = BatchNormalization()(concat_embeds)
        # 全连接+激活层+BN层
        fc1 = Dense(512, use_bias=False)(bn_concat)
        ac1 = ReLU()(fc1)
        bn1 = BatchNormalization()(ac1)
        fc2 = Dense(256, use_bias=False)(bn1)
        ac2 = ReLU()(fc2)
        bn2 = BatchNormalization()(ac2)
        fc3 = Dense(128)(bn2)
        ac3 = ReLU()(fc3)

        self.categ_inputs = categ_inputs
        self.conti_input = conti_input
        self.deep_component_outlayer = ac3

    #self.x_train_categ_poly.shape[1] --> 37
    def wide_component(self):
        # wide部分的组件
        dim = self.x_train_categ_poly.shape[1]
        self.logistic_input = Input(shape=(dim,))
        
    # X           *   W      = Y
    # (None, 165) * (165,1)  = (None,1)
    def create_model(self):
        # wide+deep
        self.deep_component()
        self.wide_component()
        if self.mode == 'wide and deep':
            out_layer = concatenate([self.deep_component_outlayer, self.logistic_input])
            inputs = [self.conti_input] + self.categ_inputs + [self.logistic_input]
        elif self.mode =='deep':
            out_layer = self.deep_component_outlayer
            inputs = [self.conti_input] + self.categ_inputs
        else:
            print('wrong mode')
            return
        
        output = Dense(1, activation='sigmoid')(out_layer)
        self.model = Model(inputs=inputs, outputs=output)

    # 训练
    # x: 训练数据的 Numpy 数组
    # y: 目标（标签）数据的 Numpy 数组
    # self.model.fit(x=None, y=None,epochs=epochs, batch_size=batch_size)
    def train_model(self, epochs=5, optimizer='adam', batch_size=128):
        # 不同结构的训练
        
        # 没有model的情况
        if not self.model:
            print('You have to create model first')
            return
        
        # 使用wide&deep的情况
        if self.mode == 'wide and deep':
            print('type wide and deep is \n',type(self.x_train_categ))
            input_data = [self.x_train_conti] +\
                         [self.x_train_categ[:, i] for i in range(self.x_train_categ.shape[1])] +\
                         [self.x_train_categ_poly]
        # 只使用deep的情况
        elif self.mode == 'deep':
            input_data = [self.x_train_conti] +\
                         [self.x_train_categ[:, i] for i in range(self.x_train_categ.shape[1])]
        else:
            print('wrong mode')
            return
        
        self.model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
        self.model.fit(input_data, self.y_train, epochs=epochs, batch_size=batch_size)

    # 评估
    def evaluate_model(self):
        if not self.model:
            print('You have to create model first')
            return

        if self.mode == 'wide and deep':
            input_data = [self.x_test_conti] +\
                         [self.x_test_categ[:, i] for i in range(self.x_test_categ.shape[1])] +\
                         [self.x_test_categ_poly]
            print('input_data len',len(input_data))
        elif self.mode == 'deep':
            input_data = [self.x_test_conti] +\
                         [self.x_test_categ[:, i] for i in range(self.x_test_categ.shape[1])]
        else:
            print('wrong mode')
            return

        loss, acc = self.model.evaluate(input_data, self.y_test)
        print(f'test_loss: {loss} - test_acc: {acc}')

    def save_model(self, filename='wide_and_deep.h5'):
        self.model.save(filename)

    # 预测
    def predict_model(self):
        if not self.model:
            print('You have to create model first')
            return

        if self.mode == 'wide and deep':
            input_data = [self.x_all_conti] +\
                         [self.x_all_categ[:, i] for i in range(self.x_all_categ.shape[1])] +\
                         [self.x_all_categ_poly]
        elif self.mode == 'deep':
            input_data = [self.x_all_conti] +\
                         [self.x_all_categ[:, i] for i in range(self.x_all_categ.shape[1])]
        else:
            print('wrong mode')
            return

        #predictions = self.model.predict(input_data, self.y_test)
        
wide_deep_net = Wide_and_Deep()
wide_deep_net.create_model()
wide_deep_net.train_model()
wide_deep_net.evaluate_model()
wide_deep_net.save_model()

1    167368
0     85686
Name: label, dtype: int64
type wide and deep is 
 <class 'numpy.ndarray'>
Train on 227748 samples
Epoch 1/5
227748/227748 [==============================] - 18s 79us/sample - loss: 66120.9220 - accuracy: 0.5511
Epoch 2/5
227748/227748 [==============================] - 16s 72us/sample - loss: 4432.5513 - accuracy: 0.5571
Epoch 3/5
227748/227748 [==============================] - 18s 78us/sample - loss: 3172.8840 - accuracy: 0.5685
Epoch 4/5
227748/227748 [==============================] - 20s 87us/sample - loss: 3072.4155 - accuracy: 0.5754
Epoch 5/5
227748/227748 [==============================] - 20s 87us/sample - loss: 2906.1982 - accuracy: 0.5812
input_data len 7
25306/1 [===================================================================================================================================================================================================================================================================================================

test_loss: 4186.578388339228 - test_acc: 0.6483837962150574


#### step three训练好的模型对召回集做预测

In [254]:
def preprocessing_recall(df_recall):
    assert df_recall.shape[1] == 13
    # 缺失值删除
    df_recall.dropna(how='any', axis=0)
    # 标签2值化
    all_data = df_recall.copy(deep=True)
    #all_data[LABEL_COLUMN] = all_data['评分'].apply(lambda x: 1 if x>=8 else 0 )
    del all_data['评分']
    
    # 类别型的label encoding
    for c in CATEGORICAL_COLUMNS:
        le = LabelEncoder()
        all_data[c] = le.fit_transform(all_data[c])

    # 类别型的列
    x_all_categ = np.array(all_data[CATEGORICAL_COLUMNS])
    print('x_all_categ\n',x_all_categ)
    print('x_all_categ type\n',type(x_all_categ))
    # 连续值的列
    x_all_conti = np.array(all_data[CONTINUOUS_COLUMNS], dtype='float64')

    # 对连续值的列做幅度缩放
    scaler = StandardScaler()
    x_all_conti = scaler.fit_transform(x_all_conti)
    x_all_poly = PolynomialFeatures(degree=2, interaction_only=True)
    x_all_categ_poly = x_all_poly.fit_transform(x_all_categ)
    return [ x_all_categ,  x_all_conti,x_all_categ_poly, all_data]

In [259]:
def get_predictions(df_recall):
    x_all_categ,  x_all_conti,x_all_categ_poly,all_data = preprocessing_recall(df_recall)
    #print('type x_all_categ',type(x_all_categ))
    input_data = [x_all_conti] +\
            [ x_all_categ[:, i] for i in range( x_all_categ.shape[1])] +\
            [ x_all_categ_poly]
    predictions = wide_deep_net.model.predict(input_data)
    pda = pd.DataFrame(predictions, columns=['predictions']) 
    #pda.head()
    all_data['likeornot'] = pda['predictions']
    all_data.head(1)
    all_data  = all_data.sort_values(by=['用户ID','likeornot'],ascending=False)
    all_data.head()
    user_count = 0
    #之前的数据是每个用户选前10
    #这次根据likeornot，每个用户选前5
    for user in all_data.用户ID.unique():
        usertemp_df = all_data[all_data['用户ID'] == user][:5]
        if user_count == 0:
            dfret = usertemp_df.copy(deep=True)
            user_count += 1
        else:
            dfret = pd.concat([dfret,usertemp_df],axis=0,ignore_index=True)
    return dfret

In [260]:
dfrecall = pd.read_csv('./dfrecallfeature.csv')
print(dfrecall.columns)
del dfrecall['主演']
dfresult = get_predictions(dfrecall)
dfresult.to_csv('./ranking.csv')

Index(['用户ID', 'user_rating_avg', 'user_rating_max', 'user_rating_min',
       'user_rating_median', '电影名', '评分', '用户评论次数_观看电影个数', '类型', '主演', '地区',
       '导演', '特色', '豆瓣网评分'],
      dtype='object')
x_all_categ
 [[  2   4  17 289 382]
 [  9   4  17 289 382]
 [  2   4  17 289 382]
 ...
 [  2   8  11 990 903]
 [  3   8  11 990 903]
 [ 15   8  11 990 903]]
x_all_categ type
 <class 'numpy.ndarray'>


In [261]:
dfresult.shape

(14655, 13)

#### 最后需要把"类型", "特色", "导演","电影名"转换回来